In [7]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from DataHelper import *
import seaborn as sns
import matplotlib

plt.rcParams["figure.figsize"] = (20,8)

sns.set_theme()

font = {'family' : 'DejaVu Sans', 'size'   : 25}

matplotlib.rc('font', **font)

In [8]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print(device)

cuda


### Prepare data

In [9]:
DATA_PATH = './data'
QUOTATIONS = ['AMZN', 'GOOG', 'AAPL', 'GM', 'TSLA', 'JNJ', 'XOM', 'AAL', 'KO', 'WMT']
QUOTATIONS = ['IBM']
FILE_SUFFIX = '.txt'
price_column = 'Close'

df = None
predict_n = 5

for quot in QUOTATIONS:
    #temp_df = pipeline_preprocessing(f"{DATA_PATH}/{quot}{FILE_SUFFIX}", predict_n=predict_n, price_column=price_column)
    temp_df, feature_names = features_pipeline(f"{DATA_PATH}/{quot}{FILE_SUFFIX}", predict_n=predict_n,
                                price_column=price_column, normalize_features=True, base_features_normalize=['Volume'])
    if df is None:
        df = temp_df
    else:
        df = df.append(temp_df)

print(len(df))
df.head()
df = df.dropna()

14760


In [10]:
df.isna().any()

Date                   False
Open                   False
High                   False
Low                    False
Close                  False
Adjusted               False
Volume                 False
Difference             False
PercentageDiff         False
Tendency               False
SMA(10)                False
SMA(20)                False
EMA(14)                False
EMA_Diff               False
SMA(20) - SMA(10)      False
LowLen                 False
RSI(14)                False
GAP                    False
RSI_Diff               False
Volume_diff            False
Next                   False
MACD                   False
MACD_diff              False
MACD_signal            False
BodyLen                False
BG_L_Band              False
BG_H_Band              False
BG_L_Band_Indicator    False
BG_H_Band_Indicator    False
ROC                    False
StochOsc               False
dtype: bool

In [11]:
from torch.utils.data import Dataset

class StocksDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.Tensor(data.values)
        self.target = torch.Tensor(target.values)
        
    def __getitem__(self, index):
        datapoint = self.data[index]
        target = self.target[index]
        return datapoint, target
        
    def __len__(self):
        return len(self.data)

y_column = 'Next'
data_columns = ['Close', 'Volume', 'Date'] + feature_names
#data_columns = ['MACD', 'MACD_diff', 'RSI(14)', 'PercentageDiff', 'GAP'] + [y_column]


dataset = df.copy()
dataset = dataset.loc[:, data_columns]

for col in dataset.columns:
    dataset[col] = dataset[col].replace({'higher':1, 'stay':0, 'lower':-1})
    

    
X = dataset.loc[:, dataset.columns != y_column]
y = dataset[y_column]

X.head()

,Close,Volume,Date,Difference,PercentageDiff,Tendency,SMA(10),SMA(20),EMA(14),EMA_Diff,...,MACD,MACD_diff,MACD_signal,BodyLen,BG_L_Band,BG_H_Band,BG_L_Band_Indicator,BG_H_Band_Indicator,ROC,StochOsc
33,7.346667,-0.970192,1962-03-21,0.105809,0.070286,-1,-0.980276,-0.979046,-0.980803,0.210844,...,0.238808,0.081335,0.217010,-0.963745,-0.979647,-0.977760,-1.0,-1.0,-0.365719,-0.375660
34,7.266667,-0.953776,1962-03-22,0.095320,-0.070885,-1,-0.981061,-0.979508,-0.981409,0.209483,...,0.236837,0.078876,0.215737,-0.979625,-0.980493,-0.977844,-1.0,-1.0,-0.411013,-0.542235
35,7.293333,-0.944920,1962-03-23,0.091041,-0.124233,-1,-0.981655,-0.979912,-0.981899,0.207081,...,0.235802,0.079002,0.214478,-0.995206,-0.981188,-0.977961,-1.0,-1.0,-0.397312,-0.498637
36,7.290000,-0.945352,1962-03-26,0.085106,-0.200472,-1,-0.982159,-0.980325,-0.982444,0.203304,...,0.234497,0.078498,0.213167,-0.983221,-0.981983,-0.978002,-1.0,-1.0,-0.409768,-0.651225
37,7.153333,-0.921376,1962-03-27,0.095734,-0.064401,-1,-0.982153,-0.980732,-0.982771,0.203066,...,0.234743,0.081565,0.212175,-0.979626,-0.982415,-0.978371,-1.0,-1.0,-0.294152,-0.455555


N_COLS = 4
plot_feature_names = data_columns.copy()

if 'Next' in plot_feature_names:
    plot_feature_names.remove('Next')
if 'Volume' in dataset.columns.values:
    plot_feature_names.append('Volume')

# get an array of subplots depending on the number of features, splitted into 3 columns
fig, axs = plt.subplots(int(np.ceil(len(plot_feature_names) / N_COLS)), N_COLS)
fig.suptitle('Values of features (scaled) filtered by class')
for i, feature in enumerate(plot_feature_names):
    sub_df = dataset[[feature, 'Next']]
    #print(f"[{i}] : {i // N_COLS}, {i % N_COLS}")
    for target_c in y.value_counts().to_dict().keys():
        filtered_df = sub_df[sub_df['Next'] == target_c]
        # plot the histogram with the 
        axs[i // N_COLS, i % N_COLS].hist(filtered_df[feature], label=f"{feature} [{target_c}]")
        axs[i // N_COLS, i % N_COLS].legend(loc='best')


plt.show()

In [12]:
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train_dataset = StocksDataset(X_train,y_train)
test_dataset = StocksDataset(X_test, y_test)

print(f"Train size : {len(train_dataset)}, test_size : {len(test_dataset)}")

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
print()

### Model

In [ ]:
import torch.nn as nn
from torch.autograd import Variable 

"""Some code adapted from https://cnvrg.io/pytorch-lstm/"""
class LSTMModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, n_layers):
        super(LSTMModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64,256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
            #nn.Softmax(1),
        )
        
        
    def forward(self, x):
        print(x.shape)
        h_0 = Variable(torch.zeros(self.n_layers, x.size(0), self.hidden_dim)) #hidden state
        c_0 = Variable(torch.zeros(self.n_layers, x.size(0), self.hidden_dim)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_dim) #reshaping the data for Dense layer next
        out = self.classifier(hn)
        return out
    
import DL_utils
import importlib
importlib.reload(DL_utils)
from DL_utils import *

def create_model(lr, input_dim):
    model = LSTMModel(input_dim=input_dim, hidden_dim=5, output_dim=1, n_layers=3)
    print(model)
    model = model.to(device)

    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    return model, optimizer, loss_fn

lr = 2e-5

model, optimizer, loss_fn = create_model(lr, input_dim = len(X.columns))
model = model.to(device)
train(train_dataloader, model, n_epochs=200, optimizer=optimizer, loss_fn=loss_fn, device=device)

In [ ]:
import DL_utils
import importlib
importlib.reload(DL_utils)
from DL_utils import *

predictions, labels = predict(test_dataloader, model, device)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(f"Test accuracy : {100 * accuracy_score(predictions, labels):.2f} %")

class_labels = ['lower', 'higher']

cf_matrix = confusion_matrix(y_true=labels, y_pred=predictions, normalize='all')
predict_df = pd.DataFrame(cf_matrix, index = [c for c in class_labels], columns = [c for c in class_labels])
ax = sns.heatmap(predict_df, annot=True)
ax.set_xlabel('Predictions')
ax.set_ylabel('True labels')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, auc


fpr, tpr, thresholds = roc_curve(labels, predictions)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color='orange', lw=4, label=f"ROC curve ({roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='blue', lw=4, linestyle='dashed')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC AUC Curve')
plt.legend(loc="best")
plt.show()